#### **1. Khôi phục các số nguyên tố từ hệ đối xứng**
Về lý thuyết, bạn hoàn toàn có thể tìm lại được các số nguyên tố $p, q, r, s$ từ các giá trị:

$ h_1 = p + q + r + s $

$ h_2 = p^2 + q^2 + r^2 + s^2 $

$ h_3 = p^3 + q^3 + r^3 + s^3 $

$ N = pqrs $

Tuy nhiên, trên thực tế việc giải yêu cầu công cụ đại số tính toán cao cấp như **SymPy** hoặc **SageMath**, và cách này chỉ hiệu quả nếu độ dài bit của các số nguyên tố không quá lớn.

---

#### **2. Ý tưởng khôi phục từ $h_1, h_2, h_3$**

Đây là một **hệ đa thức đối xứng** — ta có thể trích xuất các hệ số của đa thức đặc trưng.

##### **2.1. Định nghĩa elementary symmetric polynomials:**

Giả sử:
$$
f(x) = (x - p)(x - q)(x - r)(x - s)
$$

Khai triển ra dạng:
$$
f(x) = x^4 - a_1 x^3 + a_2 x^2 - a_3 x + a_4
$$

Với các hệ số:

- $a_1 = p + q + r + s = h_1$
- $a_2 = pq + pr + ps + qr + qs + rs$
- $a_3 = pqr + pqs + prs + qrs$
- $a_4 = pqrs = N$

Chúng ta đã biết:
- $a_1 = h_1$
- $a_4 = N$

Hai hệ số còn lại $a_2$ và $a_3$ có thể được tính ngược lại từ các công thức **Newton**:

#### 2. Newton's Identities:

$$
h_1 = a_1
$$

$$
h_2 = a_1^2 - 2a_2
$$

$$
h_3 = a_1^3 - 3a_1 a_2 + 3a_3
$$

Giải hệ để tìm $a_2$, $a_3$:

$
a_2 = \frac{a_1^2 - h_2}{2}
$

$
a_3 = \frac{h_3 - a_1^3 + 3a_1 a_2}{3}
$

---

### 3. Xây dựng và phân tích đa thức $f(x)$

Sau khi có đầy đủ các hệ số $a_1, a_2, a_3, a_4$, bạn thu được:

$$
f(x) = x^4 - a_1 x^3 + a_2 x^2 - a_3 x + a_4
$$

Bạn có thể dùng:

- `sympy.poly().factor()` để phân tích đa thức (nếu muốn nghiệm chính xác và nguyên).
- `numpy.roots()` (đơn giản nhưng độ chính xác thấp hơn) để tìm nghiệm gần đúng.

Kết quả các nghiệm sẽ chính là các số nguyên tố $p, q, r, s$.


In [ ]:
# Đọc các biến từ file output.txt
def read_variables(filename):
    variables = {}
    with open(filename, "r") as f:
        for line in f:
            line = line.strip()
            if '=' in line:
                name, value = line.split('=', 1)
                name = name.strip()
                value = value.strip()
                try:
                    variables[name] = int(value)
                except ValueError:
                    print(f"⚠️ Không thể chuyển giá trị của {name} sang số nguyên.")
    return variables

variablesL = read_variables("output.txt")
# for name, value in variablesL.items():
#     print(f"{name} = {value}")
    


44626154099651354925697068610752642661842459492769931945027538340211738148995902544351457443643808803963130274930824732652561687395268828472477422919262224
516671113554555861164166966331322883848052630063409185414998284127910160310316421085219788291486248715029393774584960034375836715001130337767354512063372620828300201147366138270597133744747341658011663632381219284289144790858167258162656417236910634201286428763727072739569460623482985066956478781223378673732


In [6]:
from Crypto.Util.number import getPrime, inverse, long_to_bytes
from sympy import symbols, Poly, factor, sqrt, simplify
from sympy import Integer as symInt


h1 = variablesL.get("h1", 0)
h2 = variablesL.get("h2", 0)
h3 = variablesL.get("h3", 0)
N = variablesL.get("N", 0)
ct = variablesL.get("ct", 0)

# Tính lại các hệ số đối xứng a1, a2, a3, a4
a1 = h1
a4 = N

a2 = (a1**2 - h2) // 2
a3 = (h3 - a1**3 + 3*a1*a2) // 3

# Xây dựng lại đa thức f(x) = x^4 - a1 x^3 + a2 x^2 - a3 x + a4
x = symbols('x')
f = x**4 - a1 * x**3 + a2 * x**2 - a3 * x + a4

print("Phân tích đa thức...")
f_poly = Poly(f)
f_factors = factor(f_poly)

print("📌 Đa thức phân tích thành:")
print(f_factors)

# Trích nghiệm (các số nguyên tố p, q, r, s ban đầu)
roots = f_poly.all_roots()

phiN = 1
for i, root in enumerate(roots, 1):
    print(f"root {i} = {root}")
    phiN = phiN * (root - 1)

e = 65537
d = int(inverse(e, phiN))
pt = pow(ct, d, N)
FLAG = long_to_bytes(pt)
print(f"pt: {FLAG}")

Phân tích đa thức...
📌 Đa thức phân tích thành:
Poly(x**4 - 44626154099651354925697068610752642661842459492769931945027538340211738148995902544351457443643808803963130274930824732652561687395268828472477422919262224*x**3 + 737411258085636785605928727269976754760256423632416271814171736535092244055086083108269913776069350194556414100976562890589008829209391953413491023861814694433264318331686951796924006512436987148444317303915584599611109745555773353892115645533199813428179134092653306762794283874721996833681920637434547376222*x**2 - 5332834060162259932206235400172019016906050650236392738935478416745288601985160591789983057166103703616406801859605076029258770448263166089500715031184682663986958095459862379447434843225149243397159823030594011273814663078702426106491776250077783022983768871441374115515710899158180396141710669189372004809172192617235875814479595381520934675261741356530568357416730882351370065815899649679888269873309611817956165435007370788078752010557186238739965762165005